In [1]:
import sys
import numpy as np
import pandas as pd
# For ease of running, make sure requestSheetFunctions.py is in the same folder as this notebook
import requestSheetFunctions as rsf

# Instructions
All functions are stored in file requestSheetFunctions.py. Make sure this file is in the same folder as this jupyter notebook. 

There are two classes: Requests and Programs. You will first generate a Request object for each target by instantiating a new Request object and then manually entering information about the target as well as desired observational strategy. Options to pull coordinates from SIMBAD are included, be sure the target name is queriable. **See below for two examples.** Once all information is inputed, you will run 2 functions within the Request object: checkForNones() and runChecks(). These will ensure that all formatting is correct. If there are errors, messages will be outputted to you. Fix all errors and rerun these functions before continuing. 

Optional but encouraged to run computeTimeRequest(), isObservable(), and determineFeasibility(). These will calculated total time required to complete request (including overheads), check how many days in the sememster the target is accessible from Keck Observatory, and perform a first order check of the feasibility of the request.

Once all Request objects are created and passing formatting tests, instantiate a Program object and give it a list of request objects. **See below for an example.** Then run three final functions: totalProgramTime(), checkAllRequests(), and writeFile(). These compute stats for your program, check formatting again, and then write out your requests to a csv. Note, the totalProgramTime() function in the Program class will not work if you haven't run computeTimeRequest() function in the Request class for each of your requests.

Finally, email your csv to jblubin@astro.ucla.edu. Also email if any questions. 

# Definitions, please read carefully. 
Reach out to jblubin@astro.ucla.edu with any questions

**exposure_time** -- (int) The requested exposure time for each observation, in seconds. For convenience, we have included a function, computeKPFExposureTime, to compute an expected exposure time to achieve a desired single measurement RV precision, in m/s, given a target's V_mag and Teff. Running this function is not required for completingthis section. If you are choosing your exposure time based on other requirements, you may simply input the value here.

**n_observations_per_visit** -- (int) The number of observations (exposures) for each visit within a single night. For example, for a triple shot put 3 here.To compute overhead, we add (N_observations_per_visit - 1)x45 secondsto your request. This is to account for readout time. The -1 is to acknowledgeThat the last observation in a series does readout during slew.

**n_unique_nights_per_semester** -- (int) The number of times to observe (visit) the star within a night. For example, if you want to observe this star once and then again later in the same night separated by X hours, put 2 here.Note, each visit will get N_observations_per_visit worth of exposures.If this value is greater than 1, you will also need to specify an intra_night_cadence.To compute overhead, we add (N_observations_per_visit x 240) seconds to your request. This is to account for average slew and acquisition time between targets, as well asglobal time sinks, like focus and calibrations

**intra_night_cadence** -- (float) The time, in hours, that you wish to be the minimum separation between visits in the same night.If N_visits_per_night is equal to 1, then this number should be set to 0.This value may be a float to allow for fractional hour separtions.

**N_unique_nights** -- (int) The number of unique nights in the semester to observe the star. Each unique night, the star will be observed N_visits_per_night times, with each visit obtaining N_observations_per_visit exposures.Note, historically, KPF is scheduled for community cadence nights on ~100 unique nights.In a given semester, the total number of nights KPF is scheduled is the maximum possible value for N_unique_nights.Once the KPF schedule is announced and programs are allocated time, PI's who have targets with N_unique_nights > unique nights scheduledwill be contacted and advised to ammend their request or accept the limiting maximum. 

**inter_night_cadence** -- (int) The time, in days, that you wish to be the minimum separation between  observations on unique nights.For example, if you wish to observe this target with at least 10 unique days separation between observations, put 10 here. Note these are calendar days, not KPF scheduled days. If no value is specified, the default value is set to 1 day. This means the target can be observed on every possible KPF scheduled night, until its requsted total observations are achieved. If you wish to only get a single observation of a target, set inter_night_cadence equal to 0.


# Example 1
Request to observe a triple shot of a star once per night. Get 50 total unique nights separated by at least 3 days.  

In [2]:
# Initialize a new Request object with the simbad resolvable star name
Example1 = rsf.Request("Sirius")
# query simbad for star info
Example1.get_star_info()

# # In this case, Sirius does not have a Gaia DR1, DR2, or DR3 name. 
# # So we will have to set a holder name in order to allow the code to work. 
# # Try commenting out this line and running, see how the code breaks. 
Example1.gaia_name = "No Gaia Name"

# Host star info
# Manual input
Example1.Vmag = 11.2
Example1.Teff = 5708

# Observation info
# Manual input
# For now, max_ExpTime must equal nominal_ExpTime
Example1.simulcal = True
Example1.nominal_ExpTime = 10
Example1.max_ExpTime = 10

# Cadence info
# Please review instructions above for info on how to fill out these paramters
Example1.n_observations_per_visit = 3
Example1.n_visits_per_night = 1
Example1.n_unique_nights_per_semester = 50
Example1.intra_night_cadence = 0.
Example1.inter_night_cadence = 3

# Run checks and calculations
Example1.computeTimeRequest()
Example1.runChecks()
if Example1.canContinue:
    print("Passed all checks.")
    Example1.isObservable()
    Example1.determineFeasibility()
else:
    print("Did not pass all checks. Make changes, then re-run.")
    


Checking for parameters still set to None.

Checking for correct data types.

Checking for formatting.

All looks good for this request! 

Passed all checks.
This target is observable for a total of [72] days this semester.
This target rises on day [0] and sets on day [72] of the semester. 

Your program may not be feasible. With your desired inter-night cadence and desired unique night visits, combined with this target's accessibility this semester, there is likely not enough unique nights in the semester to complete this request. Consider ammending your request, or accept that it may not be able to be fully completed. 



# Example 2
Request to observe a star twice per night, separated by at least 3 hours. Get 25 unique nights at nightly cadence.

In [3]:
Example2 = rsf.Request("HD 219134")
Example2.get_star_info()

# Host star info
Example2.Vmag = 5.8
Example2.Teff = 5000

#Observation info
Example2.simulcal = False
Example2.nominal_ExpTime = 300
Example2.max_ExpTime = 300

#Cadence info
Example2.n_observations_per_visit = 1
Example2.n_visits_per_night = 2
Example2.n_unique_nights_per_semester = 25
Example2.intra_night_cadence = 3.0
Example2.inter_night_cadence = 1

# run checks and calculations
Example2.computeTimeRequest()
Example2.runChecks()
if Example1.canContinue:
    print("Passed all checks.")
    Example2.isObservable()
    Example2.determineFeasibility()
else:
    print("Did not pass all checks. Make changes, then re-run.")

Checking for parameters still set to None.

Checking for correct data types.

Checking for formatting.

All looks good for this request! 

Passed all checks.
This target is observable for a total of [180] days this semester.
This target rises on day [0] and sets on day [180] of the semester. 

To first order checks, this target's request is feasible. This is not a guarentee that it will actually be completed to 100% of the request.


# Additional Targets
This notebook shows only two examples but please add more cells below to build Request objects for each of your targets in your program. 

Only when each target has a Request object describing its desired observational strategy, then move on to build the Program object, as exampled below.

In [4]:
# place all Request Objects into a new Program object
myProgram = rsf.Program([Example1, Example2])

# Your 3 letter initials here
# this is used for easy sorting of targets within programs and tracking overall time accounting
myProgram.initials = 'XYZ'
myProgram.semester = '2024A' # Do not change

path = # YOUR SAVE PATH HERE
myProgram.savefile = path + "YourLastNameHERE_KPFCC_2024A.csv" #suggested filename

# run final time accounting, final checks, then write the csv file
myProgram.totalProgramTime()
myProgram.checkAllRequests()
myProgram.writeFile()

This program requires an allocation of [2.5] nights to be feasible.
Kepler-10,Gaia_DR2_2132155017099178624,TIC377780790,JL,285.68,50.24,-18.483,41.382,J2000,-1.5,8000,10,10,True,150,3,1,0.0,50,3,51000,14.17

HD 219134,Gaia_DR2_2009481748875806976,TIC283722336,JL,348.32,57.17,2074.414,294.452,J2000,5.8,5000,300,300,False,50,1,2,3.0,25,1,39000,10.83



# Last instruction
Please email your csv to jblubin@astro.ucla.edu

Please feel encouraged to also describe your experience running this notebook. Is it easy or difficult? Is it intuitive or confusing? Is there additional information about your program and your requests that you feel is not captured within this framework? Any and all comments are welcome. We will be reaching out with a more formal survey asking for feedback on your use of this notebook. 